In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_5/models/B5R2b5_AN_1FC_fold5_1.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [4]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image1/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [5]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Validationdf_fold5.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(656, 25)
(656, 25)
Normal:  (429, 25)
Abnormal:  (227, 25)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,...,left,top,width,height,Rleft,Rtop,Rwidth,Rheight,FileName,filename
0,0,0,34,34,42,P5,P52,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,94.0,80.0,572.0,585.0,0.104950,0.089112,0.633663,0.651394,AB01 P5-2 C042.JPG,AB01 P5-2 C042.JPG
1,1,1,35,35,42,P1,P1,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,231.0,134.0,544.0,380.0,0.229956,0.149621,0.541516,0.422222,AB02 P1 C042.JPG,AB02 P1 C042.JPG
2,2,2,36,36,42,P2,P2,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,238.0,140.0,497.0,549.0,0.237177,0.155682,0.494585,0.610101,AB02 P2 C042.JPG,AB02 P2 C042.JPG
3,3,3,37,37,42,P5,P51,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,318.0,160.0,494.0,601.0,0.318902,0.177904,0.494545,0.668687,AB02 P5-1 C042.JPG,AB02 P5-1 C042.JPG
4,4,4,38,38,42,P4,P42,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,300.0,143.0,474.0,621.0,0.302369,0.159722,0.477148,0.690909,AB02 P4-2 C042.JPG,AB02 P4-2 C042.JPG


In [6]:
# dataframe.info()

In [6]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 656 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [7]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [8]:
vector_array = np.array(vectorlist)
vector_array

array([[-0.09742121, -0.1386593 , -0.07530506, ..., -0.13178509,
        -0.11908072, -0.08054256],
       [-0.16551919, -0.14817028,  0.02555851, ..., -0.08719883,
         0.0390898 , -0.10288659],
       [-0.18043953, -0.09832229,  0.25279191, ..., -0.10431186,
        -0.00535067, -0.12257629],
       ...,
       [ 0.42461812,  0.55069864, -0.21325192, ...,  0.42621693,
        -0.11533031,  0.431281  ],
       [ 0.46285176,  0.47846794, -0.16856532, ...,  0.59618658,
        -0.13356361,  0.82279915],
       [ 0.80353427,  0.77031791, -0.1727947 , ...,  0.46429279,
        -0.08342023,  0.49844059]])

In [9]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,-0.097421,-0.138659,-0.075305,0.004809,-0.166975,-0.162871,0.017836,-0.181519,-0.156809,-0.188706,...,-0.115136,-0.129576,-0.041674,-0.082114,-0.050849,-0.167695,-0.061123,-0.131785,-0.119081,-0.080543
1,-0.165519,-0.148170,0.025559,0.191055,-0.116192,0.172783,0.086269,-0.170169,-0.192331,-0.205487,...,-0.118659,0.089009,0.042513,-0.143221,0.183026,0.171876,-0.182756,-0.087199,0.039090,-0.102887
2,-0.180440,-0.098322,0.252792,0.377349,-0.201334,0.028657,0.169927,-0.229849,-0.202207,-0.203773,...,-0.085335,-0.054017,0.008280,-0.146328,0.013374,-0.007976,-0.080757,-0.104312,-0.005351,-0.122576
3,-0.166250,-0.122344,0.033003,0.317674,-0.237068,-0.036622,0.080454,-0.214761,-0.218280,-0.223394,...,-0.119591,0.011199,0.046729,-0.164173,-0.007704,-0.017912,-0.155263,-0.118105,0.006027,-0.138694
4,-0.153115,-0.111383,0.023886,0.371484,-0.234303,0.068199,0.113064,-0.232416,-0.214711,-0.209452,...,-0.111249,0.026272,0.056392,-0.143647,0.130388,0.013063,-0.183605,-0.096126,-0.029792,-0.133632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,0.585565,0.864469,-0.142827,-0.026553,0.704250,-0.195384,-0.141776,0.750391,0.811801,0.785264,...,0.541082,-0.192855,-0.167410,0.932088,-0.057671,-0.040488,0.140646,1.054485,-0.160982,0.827420
652,1.179774,0.748884,-0.174159,-0.137199,0.460761,-0.107073,-0.188199,0.584849,0.655806,0.500271,...,0.768548,-0.156593,-0.123671,0.742824,-0.101403,-0.117468,0.103894,1.278070,-0.086840,0.959453
653,0.424618,0.550699,-0.213252,0.002048,0.424195,-0.173825,-0.166131,0.504484,0.366906,0.160027,...,0.443101,-0.197879,-0.163162,0.525251,-0.090035,-0.211143,-0.061633,0.426217,-0.115330,0.431281
654,0.462852,0.478468,-0.168565,-0.157872,0.423792,-0.219667,-0.172214,0.419736,0.326973,-0.152354,...,0.266797,-0.188201,-0.163066,0.458652,-0.157790,-0.205812,-0.106637,0.596187,-0.133564,0.822799


In [10]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [11]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,-0.097421,-0.138659,-0.075305,0.004809,-0.166975,-0.162871,0.017836,-0.181519,-0.156809,-0.188706,...,-0.050849,-0.167695,-0.061123,-0.131785,-0.119081,-0.080543,Abnormal,P52,FP-C,AB01
1,-0.165519,-0.148170,0.025559,0.191055,-0.116192,0.172783,0.086269,-0.170169,-0.192331,-0.205487,...,0.183026,0.171876,-0.182756,-0.087199,0.039090,-0.102887,Abnormal,P1,FP-A,AB02
2,-0.180440,-0.098322,0.252792,0.377349,-0.201334,0.028657,0.169927,-0.229849,-0.202207,-0.203773,...,0.013374,-0.007976,-0.080757,-0.104312,-0.005351,-0.122576,Abnormal,P2,FP-A,AB02
3,-0.166250,-0.122344,0.033003,0.317674,-0.237068,-0.036622,0.080454,-0.214761,-0.218280,-0.223394,...,-0.007704,-0.017912,-0.155263,-0.118105,0.006027,-0.138694,Abnormal,P51,FP-C,AB02
4,-0.153115,-0.111383,0.023886,0.371484,-0.234303,0.068199,0.113064,-0.232416,-0.214711,-0.209452,...,0.130388,0.013063,-0.183605,-0.096126,-0.029792,-0.133632,Abnormal,P42,FP-B,AB02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
651,0.585565,0.864469,-0.142827,-0.026553,0.704250,-0.195384,-0.141776,0.750391,0.811801,0.785264,...,-0.057671,-0.040488,0.140646,1.054485,-0.160982,0.827420,Normal,P61,FP-C,Normal
652,1.179774,0.748884,-0.174159,-0.137199,0.460761,-0.107073,-0.188199,0.584849,0.655806,0.500271,...,-0.101403,-0.117468,0.103894,1.278070,-0.086840,0.959453,Normal,P62,FP-C,Normal
653,0.424618,0.550699,-0.213252,0.002048,0.424195,-0.173825,-0.166131,0.504484,0.366906,0.160027,...,-0.090035,-0.211143,-0.061633,0.426217,-0.115330,0.431281,Normal,P71,FP-D,Normal
654,0.462852,0.478468,-0.168565,-0.157872,0.423792,-0.219667,-0.172214,0.419736,0.326973,-0.152354,...,-0.157790,-0.205812,-0.106637,0.596187,-0.133564,0.822799,Normal,P72,FP-D,Normal


In [12]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVvalidation_AN_RF_AN_fold5_1.csv')